In [24]:
import os
import collections
import json
from PIL import Image
import PIL
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tq
import pickle
with open("test_rectangles.p","rb") as f:
        my_list = pickle.load(f)
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from keras.utils.visualize_util import plot
from keras.optimizers import *
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
import csv
from sklearn.metrics import log_loss
from PIL import Image
import os
import random

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [22]:
NUM_IMGS = 1000
print(my_list[50])

{'rectangles': [(900.0, 451.7647058823529, 135.0, 112.94117647058823)], 'path': './test_stg1/img_00380.jpg'}


In [30]:
test_path = './test/test/'
crop_path = './crop_test/'
for element in tq(my_list):
    rec = element['rectangles']
    if rec != []:
        for i,r in enumerate(rec):
            r = list(r)
            x = r[0]
            y = r[1]
            w = r[2]
            h = r[3]  
            img_name = element['path'].split('/')[-1]
            img = Image.open(test_path+path)
            output_name = crop_path+str(i)+'_'+img_name
            img2 = img.crop((x, y, x+w, y+h))#.resize((size), PIL.Image.ANTIALIAS)          
            img2.save(output_name) 
    else:
        img_name = element['path'].split('/')[-1]
        img = Image.open(test_path+path)
        output_name = crop_path+str(i)+'_'+img_name
        img2.save(output_name)

In [33]:
num_images = str(NUM_IMGS - len(os.listdir(crop_path)))
print(num_images)
img_name_list = os.listdir(crop_path)
print(img_name_list[:10])

-212
['0_img_00007.jpg', '0_img_00009.jpg', '0_img_00018.jpg', '0_img_00027.jpg', '0_img_00030.jpg', '0_img_00040.jpg', '0_img_00046.jpg', '0_img_00053.jpg', '0_img_00071.jpg', '0_img_00075.jpg']


In [35]:
from keras.models import load_model
model = load_model('model.h5')

In [106]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import collections
predictions = []
test_id = []
predic = collections.defaultdict(list)
for img_name in img_name_list:
    image_path = crop_path+img_name
    img = img_to_array(Image.open(image_path))
    img = img.reshape((1,) + img.shape)
    results = model.predict(img)
    obj = np.array(results[0])
    obj = np.hstack((np.array(img_name[2:]).T,obj))
    predic[img_name[2:]].append(obj)

In [109]:
keys = [key for key in predic.keys()]
for key in keys[:10]:
    print(predic[key])

[array(['img_04624.jpg', '0.6429305672645569', '0.06413475424051285',
       '0.025188583880662918', '0.044576577842235565',
       '0.011613131500780582', '0.01712697744369507',
       '0.018479248508810997', '0.17595013976097107'], 
      dtype='<U32')]
[array(['img_00383.jpg', '0.5485249161720276', '0.0810767412185669',
       '0.036442968994379044', '0.0581887885928154',
       '0.019656337797641754', '0.025365812703967094',
       '0.02564859576523304', '0.2050958126783371'], 
      dtype='<U32')]
[array(['img_00152.jpg', '0.574535608291626', '0.07225557416677475',
       '0.03280375897884369', '0.05224671959877014',
       '0.016777096316218376', '0.021733231842517853',
       '0.02451148070394993', '0.20513659715652466'], 
      dtype='<U32')]
[array(['img_04375.jpg', '0.7285491228103638', '0.04505643621087074',
       '0.014992879703640938', '0.028463490307331085',
       '0.005633574444800615', '0.008593629114329815',
       '0.009523481130599976', '0.15918737649917603'], 
   

In [158]:
from numpy import multiply
keys = [key for key in predic.keys()]
prediction_final = collections.defaultdict(list)
for key in tq(keys):
    #prediction_final[key]
    if len(predic[key]) > 1:
        v = np.ones((8,1),dtype=np.float)
        for predi in predic[key]:
            img_name = predi[0]
            predi_prob = predi[1:].astype(np.float)
            v = np.multiply(predi_prob,v.T).T
        sum = v.sum()
        w = np.array([ 1/sum for i in range(8)],dtype=np.float)
        v = np.multiply(w,v.T).T
        v = v.astype(np.str)
        img_name = img_name.reshape((1,) + img_name.shape) 
        prediction_final[key] = np.hstack((np.array(img_name),v.T[0]))
    else: 
        for predi in predic[key]:
            pred = predi[1:]
            img_name = predi[0]
            break
        prediction_final[key] = np.hstack((np.array(img_name),pred))

In [162]:
keys_final = [key for key in prediction_final.keys()]
p = []
print(len(keys_final))
for key in keys_final:
    p.append(prediction_final[key])
print(p[0])

1000
['img_04624.jpg' '0.6429305672645569' '0.06413475424051285'
 '0.025188583880662918' '0.044576577842235565' '0.011613131500780582'
 '0.01712697744369507' '0.018479248508810997' '0.17595013976097107']


In [165]:
import datetime
import pandas as pd
def create_submission(predictions, test_id, info):
    #result1 = pd.DataFrame(predictions, columns=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT','image'])
    result1 = pd.DataFrame(predictions, columns=['image','ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])
    #result1.loc['image',:] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    result1.to_csv(sub_file, index=False)
create_submission(p, test_id, '')